In [7]:

from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value


In [11]:
model = LpProblem("Maximize_Production", LpMaximize)

x_A = LpVariable("x_A", lowBound=0)
x_B = LpVariable("x_B", lowBound=0)
x_C = LpVariable("x_C", lowBound=0)

model += 120 * x_A + 100 * x_B + 150 * x_C

model += x_A <= 4 * 400
model += x_A <= 4 * 250
model += x_A <= 4 * 350
model += x_A <= 4 * 100

model += 3 * x_B <= 7 * 400
model += x_B <= 7 * 250
model += 2 * x_B <= 7 * 350
model += x_B <= 7 * 100

model += 2 * x_C <= 8 * 400
model += 2 * x_C <= 8 * 250
model += x_C <= 8 * 350
model += 3 * x_C <= 8 * 100

model.solve()

print("A :", value(x_A))
print("B :", value(x_B))
print("C :", value(x_C))
print("Максимальна сумарна вартість ", value(model.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/elizavetagribacheva/anaconda3/envs/envCV/lib/python3.13/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/7y/jzj3rvbx3875_q5lqczlj3x00000gn/T/998557e034894657a39fbff0dbcf55f1-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/7y/jzj3rvbx3875_q5lqczlj3x00000gn/T/998557e034894657a39fbff0dbcf55f1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 17 COLUMNS
At line 33 RHS
At line 46 BOUNDS
At line 47 ENDATA
Problem MODEL has 12 rows, 3 columns and 12 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-12) rows, 0 (-3) columns and 0 (-12) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 158000
After Postsolve, objective 158000, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 158000 - 0 iterations time 0.002, Presolve 0.00
Op

In [13]:
coal_supply = [400, 250, 350, 100]
ratios = {
    'A': [1/4, 1/4, 1/4, 1/4],
    'B': [3/7, 1/7, 2/7, 1/7],
    'V': [2/8, 2/8, 1/8, 3/8]
}
prices = {'A': 120, 'B': 100, 'V': 150}


In [14]:
prob_F1 = LpProblem("Maximize_Revenue", LpMaximize)
x = LpVariable.dicts("x", ['A', 'B', 'V'], lowBound=0)

prob_F1 += lpSum([prices[i] * x[i] for i in ['A', 'B', 'V']])

for j in range(4):
    prob_F1 += lpSum([ratios[i][j] * x[i] for i in ['A', 'B', 'V']]) <= coal_supply[j]

prob_F1.solve()
F1_max = value(prob_F1.objective)

print(f"Максимальна вартість (F1): {F1_max} грн")
print(f"Оптимальний випуск для F1: A={x['A'].varValue}, B={x['B'].varValue}, V={x['V'].varValue}")

delta_F1 = 0.05 * F1_max
prob_F2 = LpProblem("Maximize_Production", LpMaximize)
y = LpVariable.dicts("y", ['A', 'B', 'V'], lowBound=0)

prob_F2 += lpSum([y[i] for i in ['A', 'B', 'V']])

for j in range(4):
    prob_F2 += lpSum([ratios[i][j] * y[i] for i in ['A', 'B', 'V']]) <= coal_supply[j]
prob_F2 += lpSum([prices[i] * y[i] for i in ['A', 'B', 'V']]) >= F1_max - delta_F1

prob_F2.solve()
print(f"\nМаксимальний випуск (F2) при F1 >= {F1_max - delta_F1} грн: {value(prob_F2.objective)} тис. т")
print(f"Оптимальний випуск: A={y['A'].varValue}, B={y['B'].varValue}, V={y['V'].varValue}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/elizavetagribacheva/anaconda3/envs/envCV/lib/python3.13/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/7y/jzj3rvbx3875_q5lqczlj3x00000gn/T/08eeafce32fa4834909b5301bd2ee763-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/7y/jzj3rvbx3875_q5lqczlj3x00000gn/T/08eeafce32fa4834909b5301bd2ee763-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 25 RHS
At line 30 BOUNDS
At line 31 ENDATA
Problem MODEL has 4 rows, 3 columns and 12 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (-1) rows, 3 (0) columns and 9 (-3) elements
0  Obj -0 Dual inf 370 (3)
1  Obj 70000
Optimal - objective value 70000
After Postsolve, objective 70000, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 70000 - 1 iterations time 0.002, Presolve 0.00
Option for printing